## Linear Congruential Generator
$$
x_{t+1}=ax_t +b \pmod{m}
$$
- _Inferring a Sequence Generated by a Linear Congruence_. Joan Boyar, 1983. doi: [10.1007/978-1-4757-0602-4_32](https://doi.org/10.1007/978-1-4757-0602-4_32)
- _Cracking RNGs: Linear Congruential Generators._ Jarosław Jedynak, 2017 [tailcall.net/blog/cracking-randomness-lcgs/](https://tailcall.net/blog/cracking-randomness-lcgs/)


In [32]:
from Crypto.Util.number import getPrime, getRandomInteger, inverse
from Crypto.Random.random import choice
import math

In [33]:
class LCG():
    
    def __init__(self, **kwargs):
        primes = [getPrime(8) for i in range(4)]
        if "m" in kwargs:
            self.m = kwargs["m"]
        else:
            self.m = 3*choice(primes)*math.prod(primes)
        if "a" in kwargs:
            self.a = kwargs["a"]
        else:
            self.a = 3*choice(primes)*math.prod(primes)+1
        if "b" in kwargs:
            self.b = kwargs["b"]
        else:
            self.b = getRandomInteger(16)
        if "seed" in kwargs:
            self.x = kwargs["seed"]%self.m
        else:
            self.x = 0
            
            
    def seed(self, seed:int):
        self.x = seed % self.m
        
    def next(self):
        self.x = (self.a * self.x + self.b) % self.m
        return self.x

In [34]:
lcg = LCG(seed=getRandomInteger(4))

In [35]:
# gather initial samples, compute differences

numSamples = 20
X=[]
for i in range(numSamples):
    X.append(lcg.next())
Y = [x2 - x1 for (x2, x1) in zip(X[1:], X[:-1])]

print(X)
print(Y)

[594063352832, 23070520497, 974725375690, 761324045729, 726718466955, 870908639368, 1193894562968, 351824301414, 1032401727388, 547922968208, 242239960215, 115352703409, 167261197790, 397965443358, 807465440113, 51909251714, 819000750843, 421036064818, 201867129980, 161493946329]
[-570992832335, 951654855193, -213401329961, -34605578774, 144190172413, 322985923600, -842070261554, 680577425974, -484478759180, -305683007993, -126887256806, 51908494381, 230704245568, 409499996755, -755556188399, 767091499129, -397964686025, -219168934838, -40373183651]


To find the modulus, compute a function $f(x)\equiv 0 \pmod{m}$, e.g.
$$
f_{i}=𝑦_{i+2}𝑦_{i}−𝑦_{i+1}𝑦_{i+1}
$$
In probability, a sequence of these values will have $gcd(f)=m$.

In [36]:
# find modulus

from functools import reduce
# reduce not necessary from python 3.9, just gcd(*f)
f=[]
for i in range(numSamples-3):
    f.append(Y[i+2]*Y[i] - Y[i+1]*Y[i+1])

print('computed modulus:')
m_hat = []
for i in range(1,len(f)):
    m_hat.append( reduce(math.gcd, f[:i]) )
    print(m_hat[-1])


computed modulus:
-783796333593922513978314
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341
1343851936341


In [37]:
# check:
print(f'hidden modulus: {lcg.m}')
print(f'ratio: {m_hat[-1]//lcg.m}')

hidden modulus: 1343851936341
ratio: 1


In [38]:
# find the multiplier, a
a_hat = []
for i in range(len(m_hat)):
    a_hat.append( Y[1]*inverse(Y[0],m_hat[i]) % m_hat[i] )
    print(a_hat[-1])

-783796333592970859123121
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572
1286175887572


In [39]:
# check:
print(f'hidden multiplier: {lcg.a}')

hidden multiplier: 1286175887572


In [40]:
# finally, the increment
b_hat = []
for i in range(len(a_hat)):
    b_hat.append( X[1]-a_hat[i]*X[0] %m_hat[i] )
    print(b_hat[-1])

218453059579140558042235
50488
50488
50488
50488
50488
50488
50488
50488
50488
50488
50488
50488
50488
50488
50488


In [41]:
# check:
print(lcg.b)

50488


In [44]:
# check generated sequence
lcg_hat = LCG(m=m_hat[-1], a=a_hat[-1], b=b_hat[-1], seed=X[0])
X_hat=[]
for i in range(numSamples-1):
    X_hat.append(lcg_hat.next())
print(X_hat)
print(X[1:])

[23070520497, 974725375690, 761324045729, 726718466955, 870908639368, 1193894562968, 351824301414, 1032401727388, 547922968208, 242239960215, 115352703409, 167261197790, 397965443358, 807465440113, 51909251714, 819000750843, 421036064818, 201867129980, 161493946329]
[23070520497, 974725375690, 761324045729, 726718466955, 870908639368, 1193894562968, 351824301414, 1032401727388, 547922968208, 242239960215, 115352703409, 167261197790, 397965443358, 807465440113, 51909251714, 819000750843, 421036064818, 201867129980, 161493946329]
